# Подготовка данных пациентов

In [20]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
import transformers
from tqdm import notebook
from sklearn.model_selection import train_test_split

## Загрузка данных

In [7]:
file_path = 'chd — 100.xlsx'

In [9]:
patient_data = pd.read_excel(file_path)

In [10]:
patient_data.head()

,admittion,department,discharge,sex,height,weight,BMI,BSA,birth,Операции (все в ИБ),...,Количество выб. из ОРИТ,К.дней в ОРИТ (всего),"ИВЛ, час. в ОРИТ (суммарно)",Инф. осложнения в ОРИТ,Назн. преп. в ОРИТ,target,Непосред. причина смерти,ЭхоКГ (Из Эпикр. вып.),ЭКГ (Из Эпикр. вып.),Назначения при выписке
0,2016-12-12,ehn,2017-01-10,m,76,9.70,111.27,0.46,02.01.2016,12.12.2016: (Откр./ИК) Перевязка ранее наложен...,...,2,20,252,01.01.2017: пневмония,"Адреналина г\хл 0,1% 1мл №5; Аксетин 750мг №10...",recovery,NaN,"ЭхоКГ ВПС (02.12.2016 14:37:22, врач Неталиева...",ЭКГ (02.12.2016 16:43:25)\nРитм сердца синусов...,
1,2017-01-13,rhn,2017-02-01,f,67,7.34,89.67,0.37,02.02.2016,18.01.2017: (Откр./ИК) Радикальная коррекция д...,...,1,2,0,NaN,"Адреналина г\хл 0,1% 1мл №5; Аксетин 750мг №1;...",recovery,NaN,"от 08.08.2016, врач Плахова: ЛЖ: не увеличен, ...",Ритм сердца синусовый. Частота сердцебиений в ...,На 3 мес: верошпирон 12 мг 2р в 17-20ч под наб...
2,2017-01-17,ehn,2017-02-09,m,74,8.90,103.46,0.43,21.02.2016,19.01.2017: (Откр./ИК) Перевязка ранее наложен...,...,1,2,3,21.01.2017: инфекция ВДП,"Адреналина г\хл 0,1% 1мл №5; Дексамед 4мг/мл 1...",recovery,NaN,"ЭхоКГ ВПС (17.01.2017 16:41:49, врач Неталиева...","ЭКГ (17.01.2017 17:30:48)\nритм синусовый, отк...","фуросемид по 4 мг * 3 раза в день (14:00, 22:0..."
3,2017-01-20,ehn,2017-02-21,f,67,6.97,85.15,0.36,02.03.2016,23.01.2017: (Откр./ИК) Наложение двустороннего...,...,1,3,24,26.01.2017: -,Аксетин 750мг №1; Аминовен Инфант 10% 100мл №1...,recovery,NaN,"21.12.2016\nЛЖ - гипоплазирован, КДР-1.1-1.2. ...","Ритм синусовый, ЭОС отклонена влево",1. Варфарин по 1\2 таб. 1 р\день (++--).под ко...
4,2017-02-13,onik,2017-03-01,m,82,9.29,102.59,0.46,08.03.2016,15.02.2017: (Откр./ИК) Реконструкция путей отт...,...,1,5,48,20.02.2017: -,"Адреналина г\хл 0,1% 1мл №5; Аксетин 750мг №1;...",recovery,NaN,"08.02.17 (Неталиева) ЛП, ЛЖ - не увеличены, КД...",14.02.17 Ритм синусовый. ЧСС 120/мин. ЭОС откл...,"педиатра, невропатолога по месту жительства. ..."


In [11]:
patient_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 43 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   admittion                                        99 non-null     datetime64[ns]
 1   department                                       99 non-null     object        
 2   discharge                                        99 non-null     datetime64[ns]
 3   sex                                              99 non-null     object        
 4   height                                           99 non-null     int64         
 5   weight                                           99 non-null     float64       
 6   BMI                                              99 non-null     float64       
 7   BSA                                              99 non-null     float64       
 8   birth                                     

In [12]:
patient_data.describe()

,height,weight,BMI,BSA,"Время ПерАо, мин.",Осложнения опер.,Количество пост. в ОРИТ,Количество выб. из ОРИТ,К.дней в ОРИТ (всего),"ИВЛ, час. в ОРИТ (суммарно)"
count,99.000000,99.000000,99.000000,99.000000,55.000000,0.0,99.000000,99.000000,99.000000,99.000000
mean,69.161616,7.208677,85.899394,0.372222,56.927273,NaN,0.818182,0.777778,3.757576,61.070707
std,4.841797,1.450013,15.748346,0.051221,36.493964,NaN,0.502314,0.506197,5.730496,171.159924
min,58.000000,4.400000,44.810000,0.210000,4.000000,NaN,0.000000,0.000000,0.000000,0.000000
25%,66.000000,6.090000,72.510000,0.330000,31.000000,NaN,1.000000,0.500000,1.000000,0.000000
50%,69.000000,7.255000,86.710000,0.370000,51.000000,NaN,1.000000,1.000000,2.000000,11.000000
75%,72.500000,8.030000,95.160000,0.400000,76.500000,NaN,1.000000,1.000000,4.000000,48.000000
max,85.000000,11.100000,123.090000,0.520000,179.000000,NaN,3.000000,3.000000,35.000000,1512.000000


Успешно создан датасет. Необходимо изменить тип данных некоторых признаков, убрать пропуски.

## Предобработка

In [19]:
main_dataset = pd.DataFrame()

In [22]:
train, test = train_test_split(patient_data, test_size=0.2)

In [ ]:
main_dataset[['height', 'weight', 'BMI', 'BSA', '']]

### Изменение типа признаков

In [13]:
patient_data['birth'] = pd.to_datetime(patient_data['birth'])
patient_data['Дата опер.'] = pd.to_datetime(patient_data['Дата опер.'])
patient_data['Поступил в ОРИТ'] = pd.to_datetime(patient_data['Поступил в ОРИТ'])
patient_data['Выбыл из ОРИТ'] = pd.to_datetime(parient_data['Выбыл из ОРИТ'])

ParserError: String does not contain a date:  

### OHE

#### sex

In [23]:
sex_OHE = OneHotEncoder(drop='first').fit(train['sex'].values.reshape(-1, 1))

In [25]:
main_dataset['sex'] = sex_OHE.transform(train['sex'].values.reshape(-1, 1)).toarray()

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

#### department

In [46]:
department_ohe = OneHotEncoder().fit(patient_data['department'].values.reshape(-1, 1))
department_ohe.categories_[0]

array(['ehn', 'onik', 'rhn'], dtype=object)

In [51]:
patient_data[department_ohe.categories_[0]] = pd.DataFrame(department_ohe.transform(patient_data['department'].values.reshape(-1, 1)).toarray())

In [52]:
patient_data = patient_data.drop('department', axis=1)

### Feature Engineering

In [ ]:
patient_data['age'] = 

### Обработка текста